In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **DOWNLOAD DATASET**

In [ ]:
!pip install gdown

In [ ]:
!gdown --id 1ymDYrGs9DSRicfZbSCDiOu0ikGDh5k6S -O FSC_147/myfile.zip

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1ymDYrGs9DSRicfZbSCDiOu0ikGDh5k6S
From (redirected): https://drive.google.com/uc?id=1ymDYrGs9DSRicfZbSCDiOu0ikGDh5k6S&confirm=t&uuid=beb0c8e7-2bf9-417b-8b5f-261371b13192
To: /Project/FSC_147/myfile.zip
100%|██████████████████████████████████████| 1.53G/1.53G [00:22<00:00, 69.1MB/s]


In [ ]:
!unzip -o FSC_147/myfile.zip -d  FSC_147

Archive:  FSC_147/myfile.zip
  inflating: FSC_147/gt_density_map_adaptive_384_VarV2/1050.npy  
  inflating: FSC_147/gt_density_map_adaptive_384_VarV2/1053.npy  
  inflating: FSC_147/gt_density_map_adaptive_384_VarV2/1061.npy  
  inflating: FSC_147/gt_density_map_adaptive_384_VarV2/1074.npy  
  inflating: FSC_147/gt_density_map_adaptive_384_VarV2/1077.npy  
  inflating: FSC_147/gt_density_map_adaptive_384_VarV2/1084.npy  
  inflating: FSC_147/gt_density_map_adaptive_384_VarV2/1085.npy  
  inflating: FSC_147/gt_density_map_adaptive_384_VarV2/1087.npy  
  inflating: FSC_147/gt_density_map_adaptive_384_VarV2/1106.npy  
  inflating: FSC_147/gt_density_map_adaptive_384_VarV2/1123.npy  
  inflating: FSC_147/gt_density_map_adaptive_384_VarV2/1130.npy  
  inflating: FSC_147/gt_density_map_adaptive_384_VarV2/1246.npy  
  inflating: FSC_147/gt_density_map_adaptive_384_VarV2/1282.npy  
  inflating: FSC_147/gt_density_map_adaptive_384_VarV2/1306.npy  
  inflating: FSC_147/gt_density_map_adaptive_38

In [ ]:
!rm  /content/drive/MyDrive/CS331/myfile.zip

# **IMPORT MODULES**

In [ ]:
!pip install opencv-contrib-python numpy tqdm transformers torch torchvision

In [ ]:
import os
import json
import cv2
import torch
import numpy as np
from tqdm import tqdm
from collections import Counter

from PIL import Image
from transformers import AutoTokenizer, AutoModel
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode

In [ ]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-26vkwnnv
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-26vkwnnv
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.1 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=f056d7efab1843bb280c4174bf279c0cf37617586d39880b502e2be43e8b73f3
  Stored in directory: /tmp/pip-ephem-wheel-cache-d1y2nl80/wheels/35/3e/df/3d24cbfb3b6a06f17a2bfd7d1138900d4365d9028aa8f6e92f
Successfully built clip


In [ ]:
import os
import json
import random
from dataclasses import dataclass
from typing import List, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from PIL import Image

import clip

In [ ]:
from tqdm import tqdm

# **FSC-147-S**

In [ ]:
def load_categories(category_file):
    mapping = {}
    with open(category_file, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            img, cls = line.split("\t")
            mapping[img] = cls
    return mapping

In [ ]:
def write_json(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

In [ ]:
def read_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

In [ ]:
split_old = read_json('/content/drive/MyDrive/CS331/Train_Test_Val_FSC_147.json')
image_classes_S = read_json('/content/drive/MyDrive/CS331/FSC-147-S.json')

In [ ]:
splits = ['train', 'val', 'test']
split_dict = {}
for split in splits:
    split_dict[split] = [i for i in split_old[split] if i in list(image_classes_S.keys())]

write_json(split_dict, '/content/drive/MyDrive/CS331/Train_Test_Val_FSC_147_S.json')

In [ ]:
with open('/content/drive/MyDrive/CS331/ImageClasses_FSC147_S.txt', "w", encoding="utf-8") as f:
    for key, value in image_classes_S.items():
        f.write(f"{key}\t{value["class"]}\n")

# **CAPTIONING_v1**

In [ ]:
MEAN = (0.485, 0.456, 0.406)
STD = (0.229, 0.224, 0.225)


class Transform:
    def __init__(self, image_size=448):
        self.image_size = image_size

    def transform_from_PIL(self):
        return transforms.Compose([
            transforms.Resize((self.image_size, self.image_size), interpolation=InterpolationMode.BICUBIC),
            transforms.ToTensor(),
            transforms.Normalize(mean=MEAN, std=STD)
        ])


class ImageCaptioner:
    def __init__(self, model_path, device):
        self.device = device
        self.transform = Transform(image_size=448)
        self.load_model(model_path)

    def load_model(self, model_path):
        self.model = AutoModel.from_pretrained(
            model_path,
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True,
            use_flash_attn=True,
            trust_remote_code=True,
        ).to(self.device)
        self.model.eval()

        self.tokenizer = AutoTokenizer.from_pretrained(
            model_path,
            trust_remote_code=True,
            use_fast=False,
        )

    def convert_image_type(self, image):
        if isinstance(image, np.ndarray):
            image = Image.fromarray(image[..., ::-1])  # BGR → RGB
        elif isinstance(image, torch.Tensor):
            image = transforms.ToPILImage()(image)
        return image

    def tokenize_image(self, image):
        image = self.convert_image_type(image).convert("RGB")

        transform = self.transform.transform_from_PIL()
        pixel_values = transform(image).unsqueeze(0)  # shape (1, 3, 448, 448)

        return pixel_values.to(self.device).to(torch.float16)

    def caption_single(self, image, prompt: str):
        pixel_values = self.tokenize_image(image)
        generation_config = dict(max_new_tokens=64, do_sample=False)

        prefix = "<image>\n"
        response = self.model.chat(
            self.tokenizer,
            pixel_values,
            prefix + prompt,
            generation_config
        )
        return response


# PROMPT_TEMPLATE_EN = (
#     "Describe the {category} in this image in one English sentence."
#     "The sentence must start with the {category}."
#     "Mention its color, approximate shape, and where it is located in the scene."
#     '''If the {category} is not visible, respond with "The {category} is not visible in the image."'''

# )

PROMPT_TEMPLATE_EN = (
    "Describe the {category} in this image in one English sentence."
    "The sentence must start with the {category}."
    "Mention its color, approximate shape, and where it is located in the scene."
)



def describe_image_with_category(
    image_path: str,
    category: str,
    model_path: str = "OpenGVLab/InternVL2_5-1B",
    device: str = "cpu"
) -> str:

    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Cannot read image at: {image_path}")

    captioner = ImageCaptioner(model_path=model_path, device=device)
    prompt = PROMPT_TEMPLATE_EN.format(category=category)
    description = captioner.caption_single(img, prompt)
    return description

In [ ]:
if __name__ == "__main__":
    img_path = "/content/Screenshot 2025-12-18 082715.png"
    category = "stawberries"
    desc = describe_image_with_category(img_path, category)
    print("Rich description:", desc)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

configuration_internvl_chat.py: 0.00B [00:00, ?B/s]

configuration_intern_vit.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternVL2_5-1B:
- configuration_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternVL2_5-1B:
- configuration_internvl_chat.py
- configuration_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`torch_dtype` is deprecated! Use `dtype` instead!


...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/790 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/744 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Rich description: The image contains strawberries in bright red color, located on a lush green field.


In [ ]:
def load_categories(category_file):
    mapping = {}
    with open(category_file, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            img, cls = line.split("\t")
            mapping[img] = cls
    return mapping


def generate_captions_for_folder(
    folder_path: str,
    category_file: str,
    output_txt: str,
    model_path="OpenGVLab/InternVL2_5-1B",
    device="cuda:0"
):
    categories = load_categories(category_file)

    captioner = ImageCaptioner(model_path=model_path, device=device)

    image_files = sorted(
        [f for f in os.listdir(folder_path) if f.lower().endswith(("jpg", "jpeg", "png"))]
    )

    print(f"Found {len(image_files)} images.")

    with open(output_txt, "w", encoding="utf-8") as f:
        for img_name, cls in categories.items():
            img_path = os.path.join(folder_path, img_name)
            img = cv2.imread(img_path)

            if img is None:
                print(f"Skipping unreadable image: {img_name}")
                continue

            prompt = PROMPT_TEMPLATE_EN.format(category=cls)
            caption = captioner.caption_single(img, prompt)

            f.write(f"{img_name}\t{caption}\n")

            print(f"✓ {img_name} --> {caption}")

    print(f"\nAll captions saved to: {output_txt}")

In [ ]:
if __name__ == "__main__":
    folder = "/content/drive/MyDrive/CS331/FSC_147/images_384_VarV2"
    save_file = "/content/drive/MyDrive/CS331/DescribeImageClasses_FSC147_S.txt"
    category_file = "/content/drive/MyDrive/CS331/FSC_147/ImageClasses_FSC147_S.txt"

    generate_captions_for_folder(
        folder_path=folder,
        category_file=category_file,
        output_txt=save_file,
        model_path="OpenGVLab/InternVL2_5-1B",
        device="cuda:0"
    )

Found 6146 images.


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


✓ 7.jpg --> A person with dark hair, wearing a dark tank top, lies on a wooden floor surrounded by colorful bell peppers.


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


✓ 19.jpg --> The spatula in the image is silver, in the shape of a triangle, and is located on the right side of the baking tray.


...

✓ 7530.jpg --> Two men, one in camouflage and the other in a blue shirt, sit in a grassy field with a herd of deer.


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


✓ 7584.jpg --> A person with red hair is seated at a table, wearing a black top, and eating from a large bowl of soup.


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


✓ 7647.jpg --> The image features a black T-shirt with a graphic design and a white T-shirt with a green dinosaur print, both hanging on the wall.
✓ 7706.jpg --> Two boys, one in a red and white striped shirt and the other in a blue shirt, stand on either side of a table filled with red plastic cups, engaged in a game of beer pong.

All captions saved to: /content/drive/MyDrive/CS331/DescribeImageClasses_FSC147_S.txt


*Checking*

In [ ]:
a = load_categories('/content/drive/MyDrive/CS331/ImageClasses_FSC147.txt')
b = load_categories('/content/drive/MyDrive/CS331/DescribeImageClasses_FSC147.txt')

In [ ]:
a_keys = list(a.keys())
b_keys = list(b.keys())

In [ ]:
len(a_keys), len(b_keys)

(6146, 6146)

In [ ]:
a_keys == b_keys

True

# **CAPTIONING_v2**

In [ ]:
def load_categories(category_file):
    mapping = {}
    with open(category_file, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            img, cls = line.split("\t")
            mapping[img] = cls
    return mapping

In [ ]:
obj_dict = load_categories('/content/drive/MyDrive/CS331/ImageClasses_FSC147.txt')
des_dict = load_categories('/content/drive/MyDrive/CS331/DescribeImageClasses_FSC147_v2.txt')

In [ ]:
for key, value in des_dict.items():
    des_dict[key] = value.replace(obj_dict[key], 'objects')
write_json(des_dict, '/content/drive/MyDrive/CS331/DescribeObject_FSC147.txt')

# **FINETUNE CLIP**

In [ ]:
class FSC147Stage1Dataset(Dataset):
    """
    Dataset FSC-147 cho Stage 1:
    - Input: root folder chứa:
        root/
          images_384_VarV2/
          FSC_147/Train_Test_Val_FSC_147.json
          FSC_147/ImageClasses_FSC147.txt
    - Trả ra: (PIL.Image, pos_text, neg_text)
      pos_text  = class name của ảnh
      neg_text  = class name của ảnh khác (cố gắng khác class)
    """

    def __init__(self, root: str, split: str = "train", text_type: str = "class",):
        super().__init__()
        assert split in ["train", "val", "test"]
        assert text_type in ["class", "text"]
        self.root = root
        self.split = split
        self.text_type = text_type

        # Load split file
        split_path = os.path.join(root, "Train_Test_Val_FSC_147.json")
        with open(split_path, "r") as f:
            split_data = json.load(f)[split]
        self.im_list = [os.path.join(root, "images_384_VarV2", x) for x in split_data]

        # Load map: image_name -> class_name
        class_file = os.path.join(root, "ImageClasses_FSC147.txt")
        self.cls_dict = {}
        with open(class_file, "r", encoding="utf-8") as f:
            for line in f:
                name, cls = line.strip().split("\t")
                self.cls_dict[name] = cls

        # Load map: image_name -> describe_name
        describe_file = os.path.join(root, "DescribeObject_FSC147.txt")
        self.describe_dict = {}
        with open(describe_file, "r", encoding="utf-8") as f:
            for line in f:
                name, des = line.strip().split("\t")
                self.describe_dict[name] = des

        # Precompute class for each image & list of all (basename, cls)
        self.im_info = []
        for p in self.im_list:
            basename = os.path.basename(p)
            cls_name = self.cls_dict[basename]
            des_name = self.describe_dict[basename]
            self.im_info.append((p, basename, cls_name, des_name))

        print(f"[FSC147Stage1Dataset] Loaded {len(self.im_info)} images for split={split}")

    def __len__(self):
        return len(self.im_info)

    def __getitem__(self, idx: int):
        img_path, basename, cls_name, des_name = self.im_info[idx]

        # Load image
        img = Image.open(img_path).convert("RGB")

        if self.text_type == "class":
            pos_text = cls_name
        else:
            pos_text = des_name

        # Negative text = class của ảnh khác (cố gắng khác class)
        neg_text = pos_text
        trial = 0
        while neg_text == pos_text and trial < 10:
            j = random.randint(0, len(self.im_info) - 1)
            _, _, cls_j, des_j = self.im_info[j]
            neg_text = cls_j if self.text_type == "class" else des_j
            trial += 1
        # nếu sau 10 lần vẫn trùng, đành chấp nhận (hiếm)

        return img, pos_text, neg_text


def collate_stage1(batch) -> Tuple[List[Image.Image], List[str], List[str]]:
    """
    batch: list of (img, pos_text, neg_text)
    """
    images = [b[0] for b in batch]
    pos_texts = [b[1] for b in batch]
    neg_texts = [b[2] for b in batch]
    return images, pos_texts, neg_texts


In [ ]:
root = "FSC_147"
dataset = FSC147Stage1Dataset(root=root, split="train",text_type="text")
print("Dataset length:", len(dataset))

[FSC147Stage1Dataset] Loaded 3659 images for split=train
Dataset length: 3659


In [ ]:
# =========================
#  FFN & Adapter
# =========================

class ImageFFN(nn.Module):
    """
    FFN đặt trên image embedding 512-d của CLIP.
    (MLP nhỏ có BN + Dropout như trong paper)
    """
    def __init__(self, dim: int = 512, hidden_dim: int = 512):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(hidden_dim),
            nn.Dropout(0.1),
        )

    def forward(self, x):
        return self.net(x)


class TextAdapter(nn.Module):
    """
    Adapter cho text embedding 512-d của CLIP.
    """
    def __init__(self, dim: int = 512, hidden_dim: int = 512):
        super().__init__()
        self.fc1 = nn.Linear(dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, dim)
        self.dropout = nn.Dropout(0.1)
        self.bn = nn.BatchNorm1d(hidden_dim)
        self.act = nn.ReLU(inplace=True)

    def forward(self, x):
        # x: (B, D)
        h = self.act(self.fc1(x))
        h = self.dropout(h)
        h = self.act(self.fc2(h))
        h = self.bn(h)
        h = self.dropout(h)
        out = self.fc3(h)
        return out

In [ ]:
@dataclass
class Stage1Config:
    # dùng openai/clip ViT-B/16
    clip_name: str = "ViT-B/16"
    device: str = "cuda" if torch.cuda.is_available() else "cpu"

    margin: float = 1.0
    batch_size: int = 32
    num_workers: int = 4
    lr_ffn: float = 1e-4
    lr_adapter: float = 1e-4
    epochs_ffn: int = 200
    epochs_adapter: int = 200

    # Chỗ lưu checkpoint
    checkpoint_path: str = "/content/drive/MyDrive/CS331/checkpoint/richcount_stage1_fsc147.pt"

In [ ]:
# =========================
#  Stage 1 model
# =========================

class RichCountStage1(nn.Module):
    """
    Stage 1: Visual–Text Alignment trên CLIP.
    - Dùng clip.load("ViT-B/16")
    - Thêm FFN cho image
    - Thêm Adapter cho text
    - Train bằng contrastive loss
    """

    def __init__(self, cfg: Stage1Config):
        super().__init__()
        self.cfg = cfg

        # Load CLIP
        self.clip_model, self.clip_preprocess = clip.load(
            cfg.clip_name,
            device=cfg.device,
            jit=False
        )

        self.clip_model = self.clip_model.float()

        dim = self.clip_model.visual.output_dim  # thường = 512

        self.ffn = ImageFFN(dim, dim)
        self.adapter = TextAdapter(dim, dim)

        self.to(cfg.device)

    # ---------- freeze / unfreeze ----------

    def freeze_clip(self):
        for p in self.clip_model.parameters():
            p.requires_grad = False

    def freeze_ffn(self):
        for p in self.ffn.parameters():
            p.requires_grad = False

    def freeze_adapter(self):
        for p in self.adapter.parameters():
            p.requires_grad = False

    def unfreeze_ffn(self):
        for p in self.ffn.parameters():
            p.requires_grad = True

    def unfreeze_adapter(self):
        for p in self.adapter.parameters():
            p.requires_grad = True

    # ---------- encoding ----------

    def preprocess_images(self, images: List[Image.Image]) -> torch.Tensor:
        """
        images: list[PIL.Image]
        -> tensor (B, 3, 224, 224) đã normalize theo CLIP
        """
        tensors = [self.clip_preprocess(im) for im in images]
        pixel_values = torch.stack(tensors, dim=0).to(self.cfg.device)
        return pixel_values

    def preprocess_texts(self, texts: List[str]) -> torch.Tensor:
        tokens = clip.tokenize(texts).to(self.cfg.device)
        return tokens

    def get_image_emb_base(self, pixel_values: torch.Tensor) -> torch.Tensor:
        # (B, D)
        img_emb = self.clip_model.encode_image(pixel_values)
        img_emb = F.normalize(img_emb, dim=-1)
        return img_emb

    def get_text_emb_base(self, tokens: torch.Tensor) -> torch.Tensor:
        txt_emb = self.clip_model.encode_text(tokens)
        txt_emb = F.normalize(txt_emb, dim=-1)
        return txt_emb

    def encode_image_with_ffn(self, pixel_values: torch.Tensor) -> torch.Tensor:
        base = self.get_image_emb_base(pixel_values)
        out = self.ffn(base)
        return out

    def encode_text_raw(self, tokens: torch.Tensor) -> torch.Tensor:
        return self.get_text_emb_base(tokens)

    def encode_text_with_adapter(self, tokens: torch.Tensor) -> torch.Tensor:
        base = self.get_text_emb_base(tokens)
        out = self.adapter(base)
        return out

    # ---------- contrastive loss ----------

    def contrastive_loss(
        self,
        img_emb: torch.Tensor,
        pos_emb: torch.Tensor,
        neg_emb: torch.Tensor,
        margin: float
    ) -> torch.Tensor:
        """
        img_emb, pos_emb, neg_emb: (B, D)
        Loss giống Eq.(5)(6) trong paper.
        """
        pos_dist = torch.norm(img_emb - pos_emb, dim=-1)  # (B,)
        neg_dist = torch.norm(img_emb - neg_emb, dim=-1)  # (B,)

        loss_pos = pos_dist.pow(2)
        loss_neg = F.relu(margin - neg_dist).pow(2)

        loss = 0.5 * (loss_pos + loss_neg).mean()
        return loss
# =========================
#  Eval
# =========================

@torch.no_grad()
def evaluate_stage1(
    cfg: Stage1Config,
    model: RichCountStage1,
    loader: DataLoader,
    split: str = "val",
    use_adapter: bool = True
):
    model.eval()
    total = correct = 0
    dpos = dneg = 0.0

    for images, pos, neg in tqdm(loader, desc=f"Eval {split}"):
        pv = model.preprocess_images(images)
        pt = model.preprocess_texts(pos)
        nt = model.preprocess_texts(neg)

        img = model.encode_image_with_ffn(pv)

        if use_adapter:
            pos_e = model.encode_text_with_adapter(pt)
            neg_e = model.encode_text_with_adapter(nt)
        else:
            pos_e = model.encode_text_raw(pt)
            neg_e = model.encode_text_raw(nt)

        dp = torch.norm(img - pos_e, dim=-1)
        dn = torch.norm(img - neg_e, dim=-1)

        correct += (dp < dn).sum().item()
        total += dp.size(0)
        dpos += dp.sum().item()
        dneg += dn.sum().item()

    acc = correct / total
    avg_pos = dpos / total
    avg_neg = dneg / total
    gap = avg_neg - avg_pos

    print(f"[{split}] acc={acc:.4f}  d_pos={avg_pos:.4f}  d_neg={avg_neg:.4f}  gap={gap:.4f}")

    return {
        "acc": acc,
        "avg_pos": avg_pos,
        "avg_neg": avg_neg,
        "gap": gap,
    }

# =========================
#  Save / load checkpoint
# =========================

def save_checkpoint(model: RichCountStage1, cfg: Stage1Config, path: str):
    torch.save(
        {
            "state_dict": model.state_dict(),
            "config": cfg.__dict__,
        },
        path
    )
    print(f"💾 Saved checkpoint: {path}")


def load_checkpoint(model: RichCountStage1, path: str, device: str):
    ckpt = torch.load(path, map_location=device)
    model.load_state_dict(ckpt["state_dict"], strict=True)
    print(f"✅ Loaded checkpoint from: {path}")
    return ckpt


# =========================
#  Training FFN & Adapter
# =========================

def train_ffn(
    cfg: Stage1Config,
    model: RichCountStage1,
    train_loader: DataLoader,
    val_loader: DataLoader,
    ckpt_dir: str="ckpts_ffn"
    ):
    """
    Phase 1: Train FFN.
    - Freeze CLIP
    - Freeze Adapter
    - Text dùng embedding CLIP gốc
    """
    os.makedirs(ckpt_dir, exist_ok=True)

    model.freeze_clip()
    model.unfreeze_ffn()
    model.freeze_adapter()

    optimizer = torch.optim.Adam(model.ffn.parameters(), lr=cfg.lr_ffn)

    best_gap = -1e9
    best_ckpt_path = os.path.join(ckpt_dir, "best.pt")

    for epoch in range(cfg.epochs_ffn):
        model.train()
        total_loss = 0.0

        for step, (images, pos_texts, neg_texts) in enumerate(tqdm(train_loader, desc=f"FFN Epoch {epoch+1}")):
            pixel_values = model.preprocess_images(images)
            pos_tokens = model.preprocess_texts(pos_texts)
            neg_tokens = model.preprocess_texts(neg_texts)

            img_emb = model.encode_image_with_ffn(pixel_values)      # FFN có grad
            pos_emb = model.encode_text_raw(pos_tokens)              # text gốc CLIP
            neg_emb = model.encode_text_raw(neg_tokens)

            loss = model.contrastive_loss(img_emb, pos_emb, neg_emb, cfg.margin)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"[FFN] Epoch {epoch+1} Train Loss: {total_loss / len(train_loader):.4f}")
        # 🔹 EVAL DEV
        metrics = evaluate_stage1(cfg, model, val_loader, split="val", use_adapter=False)

        # # 🔹 SAVE CHECKPOINT (EVERY EPOCH)
        # ckpt_path = os.path.join(ckpt_dir, f"ffn_epoch_{epoch+1}.pt")
        # save_checkpoint(model, cfg, ckpt_path)
        save_checkpoint(model, cfg, os.path.join(ckpt_dir, "last.pt"))

        # 🔹 TRACK BEST
        if metrics["gap"] > best_gap:
            best_gap = metrics["gap"]
            # best_ckpt_path = ckpt_path
            # print(f"New BEST FFN @ epoch {epoch+1} (gap={best_gap:.4f})")
            save_checkpoint(model, cfg, best_ckpt_path)
            print(f"New BEST saved -> {best_ckpt_path} (gap={best_gap:.4f})")

    print(f"\nBest FFN checkpoint: {best_ckpt_path}")
    return best_ckpt_path


def train_adapter(
    cfg: Stage1Config,
    model: RichCountStage1,
    train_loader: DataLoader,
    val_loader: DataLoader,
    best_ffn_ckpt: str,
    ckpt_dir: str = "ckpts_adapter"):
    """
    Phase 2: Train Adapter.
    - Freeze CLIP
    - Freeze FFN
    - Only Adapter trainable
    """

    os.makedirs(ckpt_dir, exist_ok=True)

    # 🔹 LOAD BEST FFN
    load_checkpoint(model, best_ffn_ckpt, cfg.device)

    model.freeze_clip()
    model.freeze_ffn()
    model.unfreeze_adapter()

    optimizer = torch.optim.Adam(model.adapter.parameters(), lr=cfg.lr_adapter)

    best_gap = -1e9
    best_ckpt_path = os.path.join(ckpt_dir, "best.pt")

    for epoch in range(cfg.epochs_adapter):
        model.train()
        total_loss = 0.0

        for step, (images, pos_texts, neg_texts) in enumerate(tqdm(train_loader, desc=f"Adapter Epoch {epoch+1}")):
            pixel_values = model.preprocess_images(images)
            pos_tokens = model.preprocess_texts(pos_texts)
            neg_tokens = model.preprocess_texts(neg_texts)

            with torch.no_grad():
                img_emb = model.encode_image_with_ffn(pixel_values)  # fixed

            pos_emb = model.encode_text_with_adapter(pos_tokens)     # adapter train
            neg_emb = model.encode_text_with_adapter(neg_tokens)

            loss = model.contrastive_loss(img_emb, pos_emb, neg_emb, cfg.margin)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"[Adapter] Epoch {epoch+1} Train Loss: {total_loss / len(train_loader):.4f}")

        # 🔹 eval dev
        metrics = evaluate_stage1(cfg, model, val_loader, split="val", use_adapter=True)

        # # 🔹 save ckpt
        # ckpt_path = os.path.join(ckpt_dir, f"adapter_epoch_{epoch+1}.pt")
        # save_checkpoint(model, cfg, ckpt_path)

        # 🔹 track best
        if metrics["gap"] > best_gap:
            best_gap = metrics["gap"]
            # best_ckpt_path = ckpt_path
            # print(f"New BEST Adapter @ epoch {epoch+1} (gap={best_gap:.4f})")
            save_checkpoint(model, cfg, best_ckpt_path)
            print(f"New BEST saved -> {best_ckpt_path} (gap={best_gap:.4f})")

    print(f"\nBest Adapter checkpoint: {best_ckpt_path}")
    return best_ckpt_path



In [ ]:
cfg = Stage1Config()
model_infer = RichCountStage1(cfg)
model_infer = load_checkpoint(model_infer, cfg.checkpoint_path, cfg.device)

✅ Loaded checkpoint from: /content/drive/MyDrive/CS331/checkpoint/richcount_stage1_fsc147.pt


In [ ]:
# =========================
#  Main
# =========================

def main():
    cfg = Stage1Config()
    print("Using device:", cfg.device)

    root = "FSC_147"

    # train / val datasets
    train_dataset = FSC147Stage1Dataset(root=root, split="train", text_type="text")
    val_dataset = FSC147Stage1Dataset(root=root, split="val", text_type="text")
    test_dataset = FSC147Stage1Dataset(root=root, split="test", text_type="text")

    train_loader = DataLoader(
        train_dataset,
        batch_size=cfg.batch_size,
        shuffle=True,
        num_workers=cfg.num_workers,
        collate_fn=collate_stage1,
        drop_last=True,
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=cfg.batch_size,
        shuffle=False,
        num_workers=cfg.num_workers,
        collate_fn=collate_stage1,
        drop_last=False,
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=cfg.batch_size,
        shuffle=False,
        num_workers=cfg.num_workers,
        collate_fn=collate_stage1,
        drop_last=False,
    )

    # Model
    model = RichCountStage1(cfg)

    # Phase 1: train FFN
    print("========== TRAINING FFN ==========")
    best_ffn_ckpt = train_ffn(cfg, model, train_loader, val_loader, "checkpoint/ckpts_ffn")

    # Phase 2: train Adapter
    print("========== TRAINING ADAPTER ==========")
    best_adapter_ckpt = train_adapter(cfg, model, train_loader, val_loader, best_ffn_ckpt, "checkpoint/ckpts_adapter")

    # Load best adapter
    load_checkpoint(model, best_adapter_ckpt, cfg.device)

    save_checkpoint(model, cfg, cfg.checkpoint_path)
    print(f"✅ Saved final Stage 1 checkpoint to {cfg.checkpoint_path}")

    model_infer = RichCountStage1(cfg)
    load_checkpoint(model_infer, cfg.checkpoint_path, cfg.device)

    evaluate_stage1(cfg, model_infer, test_loader, split="test", use_adapter=True)


if __name__ == "__main__":
    main()


Using device: cuda
[FSC147Stage1Dataset] Loaded 3659 images for split=train
[FSC147Stage1Dataset] Loaded 1286 images for split=val
[FSC147Stage1Dataset] Loaded 1190 images for split=test
========== TRAINING FFN ==========


FFN Epoch 1: 100%|██████████| 114/114 [00:19<00:00,  5.79it/s]


[FFN] Epoch 1 Train Loss: 73.9000


Eval val: 100%|██████████| 41/41 [00:07<00:00,  5.68it/s]


[val] acc=0.5762  d_pos=7.4568  d_neg=7.4646  gap=0.0078
💾 Saved checkpoint: checkpoint/ckpts_ffn/last.pt
💾 Saved checkpoint: checkpoint/ckpts_ffn/best.pt
New BEST saved -> checkpoint/ckpts_ffn/best.pt (gap=0.0078)


...

[Adapter] Epoch 200 Train Loss: 0.1818


Eval val: 100%|██████████| 41/41 [00:07<00:00,  5.71it/s]


[val] acc=0.7854  d_pos=0.6468  d_neg=0.7543  gap=0.1075

Best Adapter checkpoint: checkpoint/ckpts_adapter/best.pt
✅ Loaded checkpoint from: checkpoint/ckpts_adapter/best.pt
💾 Saved checkpoint: checkpoint/richcount_stage1_fsc147.pt
✅ Saved final Stage 1 checkpoint to checkpoint/richcount_stage1_fsc147.pt
✅ Loaded checkpoint from: checkpoint/richcount_stage1_fsc147.pt


Eval test: 100%|██████████| 38/38 [00:06<00:00,  5.65it/s]

[test] acc=0.7664  d_pos=0.6515  d_neg=0.7370  gap=0.0856


In [ ]:
@torch.no_grad()
def compare_clip_vs_ffn_adapter(
    cfg: Stage1Config,
    model: RichCountStage1,
    loader: DataLoader,
    split: str = "dev",
):
    """
    So sánh:
    - CLIP gốc      : img = CLIP base, text = CLIP base
    - CLIP+FFN+Ada  : img = CLIP+FFN, text = CLIP+Adapter
    Dựa trên metric: acc (d_pos < d_neg), avg d_pos, avg d_neg, gap.
    """

    model.eval()
    total = 0

    # thống kê cho CLIP gốc
    correct_clip = 0
    dpos_clip = 0.0
    dneg_clip = 0.0

    # thống kê cho CLIP + FFN + Adapter
    correct_enh = 0
    dpos_enh = 0.0
    dneg_enh = 0.0

    for images, pos, neg in tqdm(loader, desc=f"Compare ({split}) CLIP vs FFN+Adapter"):
        pv = model.preprocess_images(images)
        pt = model.preprocess_texts(pos)
        nt = model.preprocess_texts(neg)

        # ----- CLIP gốc -----
        img_clip = model.get_image_emb_base(pv)
        pos_clip = model.get_text_emb_base(pt)
        neg_clip = model.get_text_emb_base(nt)

        dp_clip = torch.norm(img_clip - pos_clip, dim=-1)   # (B,)
        dn_clip = torch.norm(img_clip - neg_clip, dim=-1)   # (B,)

        correct_clip += (dp_clip < dn_clip).sum().item()
        dpos_clip += dp_clip.sum().item()
        dneg_clip += dn_clip.sum().item()

        # ----- CLIP + FFN + Adapter -----
        img_enh = model.encode_image_with_ffn(pv)
        pos_enh = model.encode_text_with_adapter(pt)
        neg_enh = model.encode_text_with_adapter(nt)

        dp_enh = torch.norm(img_enh - pos_enh, dim=-1)
        dn_enh = torch.norm(img_enh - neg_enh, dim=-1)

        correct_enh += (dp_enh < dn_enh).sum().item()
        dpos_enh += dp_enh.sum().item()
        dneg_enh += dn_enh.sum().item()

        total += dp_clip.size(0)

    # ----- tính metric -----
    acc_clip = correct_clip / total
    avg_pos_clip = dpos_clip / total
    avg_neg_clip = dneg_clip / total
    gap_clip = avg_neg_clip - avg_pos_clip

    acc_enh = correct_enh / total
    avg_pos_enh = dpos_enh / total
    avg_neg_enh = dneg_enh / total
    gap_enh = avg_neg_enh - avg_pos_enh

    print(f"\n=== [{split}] CLIP gốc ===")
    print(f"acc       = {acc_clip:.4f}")
    print(f"avg d_pos = {avg_pos_clip:.4f}")
    print(f"avg d_neg = {avg_neg_clip:.4f}")
    print(f"gap       = {gap_clip:.4f}")

    print(f"\n=== [{split}] CLIP + FFN + Adapter ===")
    print(f"acc       = {acc_enh:.4f}")
    print(f"avg d_pos = {avg_pos_enh:.4f}")
    print(f"avg d_neg = {avg_neg_enh:.4f}")
    print(f"gap       = {gap_enh:.4f}")

    return {
        "clip": {
            "acc": acc_clip,
            "avg_pos": avg_pos_clip,
            "avg_neg": avg_neg_clip,
            "gap": gap_clip,
        },
        "enhanced": {
            "acc": acc_enh,
            "avg_pos": avg_pos_enh,
            "avg_neg": avg_neg_enh,
            "gap": gap_enh,
        },
    }


def main():
    cfg = Stage1Config()
    print("Using device:", cfg.device)

    root = "FSC_147"

    # train / val datasets
    train_dataset = FSC147Stage1Dataset(root=root, split="train", text_type="text")
    val_dataset = FSC147Stage1Dataset(root=root, split="val", text_type="text")
    test_dataset = FSC147Stage1Dataset(root=root, split="test", text_type="text")

    train_loader = DataLoader(
        train_dataset,
        batch_size=cfg.batch_size,
        shuffle=True,
        num_workers=cfg.num_workers,
        collate_fn=collate_stage1,
        drop_last=True,
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=cfg.batch_size,
        shuffle=False,
        num_workers=cfg.num_workers,
        collate_fn=collate_stage1,
        drop_last=False,
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=cfg.batch_size,
        shuffle=False,
        num_workers=cfg.num_workers,
        collate_fn=collate_stage1,
        drop_last=False,
    )

    model_infer = RichCountStage1(cfg)
    load_checkpoint(model_infer, cfg.checkpoint_path, cfg.device)

    compare_clip_vs_ffn_adapter(cfg, model_infer, test_loader, split="test")


if __name__ == "__main__":
    main()


Using device: cuda
[FSC147Stage1Dataset] Loaded 3659 images for split=train
[FSC147Stage1Dataset] Loaded 1286 images for split=val
[FSC147Stage1Dataset] Loaded 1190 images for split=test


/tmp/ipykernel_1787/3085777962.py:184: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(path, map_location=device)


✅ Loaded checkpoint from: checkpoint/richcount_stage1_fsc147.pt


Compare (test) CLIP vs FFN+Adapter: 100%|██████████| 38/38 [00:10<00:00,  3.66it/s]


=== [test] CLIP gốc ===
acc       = 0.9437
avg d_pos = 1.2043
avg d_neg = 1.2717
gap       = 0.0675

=== [test] CLIP + FFN + Adapter ===
acc       = 0.7395
avg d_pos = 0.6515
avg d_neg = 0.7342
gap       = 0.0827
